In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import util
import torch
from torchviz import make_dot
from hardware_checker import get_hardware_metrics

/home/luis/bin/miniconda3/envs/hwnascim/lib/python3.10/site-packages/nni/nas/nn/pytorch/layers.py:94: RuntimeWarning: <class 'torch.nn.parameter.Buffer'> is found to be not a nn.Module, which is unexpected. It means your PyTorch version might not be supported.
  warnings.warn(f'{obj} is found to be not a nn.Module, which is unexpected. '


In [2]:
tmp = torch.load("../cifar10_lenet_params.pth", map_location="cuda:0")
# tmp.keys()
for key in tmp.keys():
    print(key)

layer_list.0.last_value
layer_list.0.bit_scale_list
layer_list.0.layer_list.0.weight
layer_list.1.last_value
layer_list.1.layer.weight
layer_list.1.layer.bias
layer_list.1.layer.running_mean
layer_list.1.layer.running_var
layer_list.1.layer.num_batches_tracked
layer_list.2.last_value
layer_list.3.last_value
layer_list.4.last_value
layer_list.4.bit_scale_list
layer_list.4.layer_list.0.weight
layer_list.5.last_value
layer_list.5.layer.weight
layer_list.5.layer.bias
layer_list.5.layer.running_mean
layer_list.5.layer.running_var
layer_list.5.layer.num_batches_tracked
layer_list.6.last_value
layer_list.7.last_value
layer_list.8.last_value
layer_list.8.bit_scale_list
layer_list.8.layer_list.0.weight
layer_list.9.last_value
layer_list.9.layer.weight
layer_list.9.layer.bias
layer_list.9.layer.running_mean
layer_list.9.layer.running_var
layer_list.9.layer.num_batches_tracked
layer_list.10.last_value
layer_list.11.last_value
layer_list.12.last_value
layer_list.12.bit_scale_list
layer_list.12.lay

In [3]:
tmp

OrderedDict([('layer_list.0.last_value', tensor([7.2205], device='cuda:0')),
             ('layer_list.0.bit_scale_list',
              tensor([[9.0000e+00, 3.9216e-03],
                      [9.0000e+00, 9.4029e-03],
                      [9.0000e+00, 2.8316e-02]], device='cuda:0')),
             ('layer_list.0.layer_list.0.weight',
              tensor([[[[ 1.3799e-01, -5.3380e-01, -1.6232e+00, -6.8974e-01,  4.3618e-01],
                        [-3.1056e-01, -8.2010e-01, -3.5214e-01, -5.8973e-01,  8.7296e-02],
                        [-8.2767e-01,  2.7090e-01,  1.8133e+00,  5.6593e-01, -6.5235e-01],
                        [ 2.5648e-01,  7.7775e-01, -3.0539e-01,  2.2004e-01,  4.0090e-01],
                        [ 2.8804e-01,  1.6467e-01, -3.8551e-01,  3.4072e-01,  2.9420e-01]],
              
                       [[ 8.4302e-01,  2.5783e-01, -7.0570e-01,  1.3511e-01,  1.0262e+00],
                        [ 3.0586e-01, -2.6178e-01,  2.5441e-01, -8.3438e-02,  3.7767e-01],
           

In [4]:
import re
key = "layer_list.0.layer_list.0.bias"
re.sub('\.[a-z]{0,3}layer_list\.\d+\.weight$', '', key)

'layer_list.0.layer_list.0.bias'

In [5]:
# import pandas as pd
# df = pd.DataFrame(tmp.items(), columns=["Key", "Value"])
# display(df)

In [6]:

class CustomModel(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        # Layers from config
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.conv3 = nn.Conv2d(16, 120, 5)
        self.dropout = nn.Dropout()
        self.fc1 = nn.Linear(120, 84)
        self.fc2 = nn.Linear(84, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))          # conv + relu
        x = self.pool(x)                   # maxpool
        x = F.relu(self.conv2(x))          # conv + relu
        x = self.pool(x)                   # maxpool
        x = F.relu(self.conv3(x))          # conv + relu
        x = torch.flatten(x, 1)            # flatten (view)
        x = self.fc1(x)
        x = self.dropout(x)
        x = F.relu(x)
        x = self.fc2(x)               # final fc
        return x

num_classes = 10

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

# Load training and test datasets
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=0)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, num_workers=0)

model = CustomModel(num_classes=100)
get_hardware_metrics(model=model, train_loader=train_loader, val_loader=test_loader)

no match for _tensor_constant0 _tensor_constant0
no match for conv1 conv1 in call_modules
no match for pool pool in call_modules
no match for conv2 conv2 in call_modules
no match for pool_1 pool in call_modules
no match for conv3 conv3 in call_modules
no match for fc1 fc1 in call_modules
no match for dropout dropout in call_modules
no match for fc2 fc2 in call_modules
run on device cuda
load state_dict
odict_keys(['conv1.weight', 'conv1.bias', 'conv2.weight', 'conv2.bias', 'conv3.weight', 'conv3.bias', 'fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias'])
[('fc', '1'), ('conv', '3'), ('fc', '2'), ('conv', '2'), ('conv', '1')]
{'fc1': 9, 'conv3': 6, 'fc2': 12, 'conv2': 3, 'conv1': 0}
Due to quantization MNSIM-2.0 does not support bias in convolution layer. Please use conv2d + batchnorm insead.
Due to quantization MNSIM-2.0 does not support bias in convolution layer. Please use conv2d + batchnorm insead.
Due to quantization MNSIM-2.0 does not support bias in convolution layer. Please use 

/home/luis/bin/miniconda3/envs/hwnascim/lib/python3.10/site-packages/torch/fx/_symbolic_trace.py:913: UserWarning: Was not able to add assertion to guarantee correct input x to specialized function. It is up to the user to make sure that your inputs match the inputs you specialized the function with.
  warnings.warn(


In [8]:
layer_config_list = []
layer_config_list.append({'type': 'conv', 'in_channels': 3, 'out_channels': 6, 'kernel_size': 5, 'padding' : 0})
layer_config_list.append({'type': 'relu'})
layer_config_list.append({'type': 'pooling', 'mode': 'MAX', 'kernel_size': 2, 'stride': 2, 'padding' : 0})
layer_config_list.append({'type': 'conv', 'in_channels': 6, 'out_channels': 16, 'kernel_size': 5, 'padding' : 0})
layer_config_list.append({'type': 'relu'})
layer_config_list.append({'type': 'pooling', 'mode': 'MAX', 'kernel_size': 2, 'stride': 2, 'padding' : 0})
layer_config_list.append({'type': 'conv', 'in_channels': 16, 'out_channels': 120, 'kernel_size': 5, 'padding' : 0})
layer_config_list.append({'type': 'relu'})
layer_config_list.append({'type': 'view'})
layer_config_list.append({'type': 'fc', 'in_features': 120, 'out_features': 84})
layer_config_list.append({'type': 'dropout'})
layer_config_list.append({'type': 'relu'})
layer_config_list.append({'type': 'fc', 'in_features': 84, 'out_features': num_classes})
layer_config_list

[{'type': 'conv',
  'in_channels': 3,
  'out_channels': 6,
  'kernel_size': 5,
  'padding': 0},
 {'type': 'relu'},
 {'type': 'pooling',
  'mode': 'MAX',
  'kernel_size': 2,
  'stride': 2,
  'padding': 0},
 {'type': 'conv',
  'in_channels': 6,
  'out_channels': 16,
  'kernel_size': 5,
  'padding': 0},
 {'type': 'relu'},
 {'type': 'pooling',
  'mode': 'MAX',
  'kernel_size': 2,
  'stride': 2,
  'padding': 0},
 {'type': 'conv',
  'in_channels': 16,
  'out_channels': 120,
  'kernel_size': 5,
  'padding': 0},
 {'type': 'relu'},
 {'type': 'view'},
 {'type': 'fc', 'in_features': 120, 'out_features': 84},
 {'type': 'dropout'},
 {'type': 'relu'},
 {'type': 'fc', 'in_features': 84, 'out_features': 10}]

In [9]:
model = CustomModel(num_classes=num_classes)
test = util.trace_model(model=model, num_classes=num_classes)
test

no match for conv1 conv1 in call_modules
no match for pool pool in call_modules
no match for conv2 conv2 in call_modules
no match for pool_1 pool in call_modules
no match for conv3 conv3 in call_modules
no match for fc1 fc1 in call_modules
no match for dropout dropout in call_modules
no match for fc2 fc2 in call_modules


[{'type': 'conv',
  'in_channels': 3,
  'out_channels': 6,
  'kernel_size': 5,
  'padding': 0},
 {'type': 'relu'},
 {'type': 'pooling',
  'mode': 'MAX',
  'kernel_size': 2,
  'stride': 2,
  'padding': 0},
 {'type': 'conv',
  'in_channels': 6,
  'out_channels': 16,
  'kernel_size': 5,
  'padding': 0},
 {'type': 'relu'},
 {'type': 'pooling',
  'mode': 'MAX',
  'kernel_size': 2,
  'stride': 2,
  'padding': 0},
 {'type': 'conv',
  'in_channels': 16,
  'out_channels': 120,
  'kernel_size': 5,
  'padding': 0},
 {'type': 'relu'},
 {'type': 'view'},
 {'type': 'fc', 'in_features': 120, 'out_features': 84},
 {'type': 'dropout'},
 {'type': 'relu'},
 {'type': 'fc', 'in_features': 84, 'out_features': 10}]

In [10]:
assert layer_config_list == test

In [11]:
input_shape = (1, 16, 8, 8)

In [ ]:
class AddTuple(nn.Module):
    def forward(self, x):
        return x[0] + x[1]
    
module = AddTuple()
util.trace_model(module, 10)

ValueError: MNSIM-2.0 only supports element sum of form 
 def forward(self, x):
                                return x[0] + x[1]
                            
See search_spaces.py class ElementSum

In [13]:
class MultiplyTuple(nn.Module):
    def forward(self, x):
        return x[0] * x[1]
module = MultiplyTuple()
# util.trace_model(module, (1,1), 10) # has to raise value error

In [14]:
class MulView(nn.Module):
    def forward(self, x):
        x0 = x[0].view(x[0].shape[0], x[0].shape[1], 1, 1)
        return x0 * x[1]
module = MulView()


util.trace_model(module, 10)

no match for getattr_1 <built-in function getattr> in call_function
no match for getattr_2 <built-in function getattr> in call_function
no match for view view in call_method


[{'type': 'element_multiply', 'input_index': [0, 1]}]

In [15]:
class SwishFn(nn.Module):
    def forward(self, x):
        return F.silu(x)
    
module = SwishFn()
util.trace_model(module, 10)

[{'type': 'Swish'}]

In [16]:
class BatchNorm2dTest(nn.Module):
    def __init__(self):
        super().__init__()
        self.bn = nn.BatchNorm2d(num_features=16)

    def forward(self, x):
        return self.bn(x)

module = BatchNorm2dTest()
util.trace_model(module, num_classes=10)

[{'type': 'batch_norm_2d',
  'num_features': 16,
  'eps': 1e-05,
  'momentum': 0.1,
  'affine': True}]

In [17]:
class ConcatTest(nn.Module):
    def forward(self, x0, x1):
        return torch.cat((x0, x1), 1)

x = (
    torch.randn(1, 8, 16, 16),
    torch.randn(1, 8, 16, 16),
)

module  = ConcatTest()
input_shape = None
util.trace_model(module, input_shape=input_shape, num_classes=10)

Concat was not validated fully. Best to avoid it


[{'type': 'concat', 'dim': 1}]